# 10. Writing mapped trajectories from Gromacs all atom (AA) trajectories

### Prerequisites
* MDAnalysis
* gsd

This example maps a system of two 12 amino acid long peptides. One amino acid residue's center of mass represents one CG bead. The all atom topology and trajecotry files can be found in `examples` directory. The mapped trajectory will be saved in gsd format to `CG_tutorial` directory. 

In [2]:
import hoomd
import numpy as np 
import hoomd.md
import gsd,gsd.hoomd
import gsd.pygsd
import hoomd.htf as htf
import tensorflow as tf
import MDAnalysis as mda

# disable GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#disable warnings
import warnings
warnings.filterwarnings('ignore')

2 Physical GPUs, 2 Logical GPUs


In this example we will use a system of two peptides which are 12 amino acids long. First we need to create the non-mass weighted mapping operator to CG the AA trajectory. Note that the mapping operator is defined only for one peptide as the peptides are identical. 

In [3]:
u = mda.Universe('htf_eg/test_topol.pdb','htf_eg/test_traj.trr')

# we select segment A to define the mapping operator. 
segA = u.select_atoms('segid A')

m = len(segA.atoms.residues) #N = num of CG beads
n = len(segA.atoms)  #AA = num of all atoms
map_mat = np.zeros((m,n))

atm_list = list(segA.atoms)
for a in atm_list:
    cgid = a.resid
    aaid = a.id
    #print(cgid,aaid)
    map_mat[cgid-1,aaid-1] =1

print('Mapping operator:', map_mat)
    

Mapping operator: [[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]


Next we need to define the CG mapping for our system using the mappng operator above. We will use the AA topology (PDB converted into a GSD file) for this. Take a look at `example 02` for a detailed description.

In [4]:
c = hoomd.context.initialize('--gpu=1')
system = hoomd.init.read_gsd(filename='htf_eg/test_topol.gsd')
c.sorter.disable()
set_rcut = 10.0

outfile = 'htf_eg/testtraj_mapped.gsd'

molecule_mapping_index = htf.find_molecules(system) 

# get number of atoms
N = sum([len(m) for m in molecule_mapping_index]) 

# get number of molecules
M = len(molecule_mapping_index) 

# get number of atoms in a molecule=MN
MN = len(molecule_mapping_index[0])

molecule_mapping = map_mat
print('atoms per mol',MN,'mapping mat',molecule_mapping.shape)
bead_number = molecule_mapping.shape[0]

cg_mapping = htf.sparse_mapping([molecule_mapping for _ in molecule_mapping_index], 
                                molecule_mapping_index, system=system)

assert cg_mapping.shape == (M * bead_number, N)

HOOMD-blue v2.9.3 CUDA (10.1) DOUBLE HPMC_MIXED SSE SSE2 
Compiled: 2021-02-03
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the following GPU(s):
 [1]  Tesla V100-PCIE-16GB  80 SM_7.0 @ 1.38 GHz, 16160 MiB DRAM, MNG
notice(2): Group "all" created containing 366 particles
Finding molecules...50.00%
atoms per mol 183 mapping mat (12, 183)


Now let's read the trajectory and generate CG bead types, indices and COMs. There are 24 residues in the system. Hence we will name each bead index by an integer scaling from 1 to 24. CG beads names will be `B + index`. A user can give any bead name of choice. We will write each mapped state to the output file in GSD format.

In [5]:
t = gsd.hoomd.open(name=outfile, mode='wb')
frame_num = 0

CGnum = M * bead_number
print('Number of CG beads: ',CGnum)
CGids = np.arange(1,CGnum+1)

#creating bead names
CGtypes = []
for i in CGids:
    CGtypes.append('B'+str(i))

for inputs, ts in htf.iter_from_trajectory(16, u, r_cut=45):
    positions = inputs[1]
    box = inputs[2]
    box_size = htf.box_size(box)
    positions = positions[:, :3]
    
    # First let's calculate the COMs of the CG groups
    #Note that each residue is grouped into one atom group and its COM is computed
    mapped_pos = tf.make_tensor_proto(htf.center_of_mass(tf.cast(positions[:,:3],dtype=tf.float64), 
                                                         tf.cast(cg_mapping,dtype=tf.float64), box_size))
    mdbox = u.dimensions
    coms = tf.make_ndarray(mapped_pos)
    
    #Let's generate snapshots of each frame and write to a GSD file 
    t.append(htf.create_frame(frame_num, CGnum, CGtypes, CGids, coms, mdbox))
    frame_num += 1

print('GSD file written')  

  0%|          | 2/801 [00:00<00:50, 15.74it/s]

Number of CG beads:  24


100%|██████████| 801/801 [00:02<00:00, 347.38it/s]

GSD file written


To see test if our mapped trajectory was written properly, we can try and read the `CG_tutorial/testtraj_mapped.gsd` file. You can see how the position of the first CG bead changes over the frames. 

In [6]:
with gsd.pygsd.GSDFile(open(outfile, 'rb')) as f:
    t = gsd.hoomd.HOOMDTrajectory(f)
    for frame,i in enumerate(t):
        pos = i.particles.position
        types = i.particles.types
        typeids = i.particles.typeid
        print(frame, pos[0])

0 [ 35.35711  -27.433498  -7.809485]
1 [-34.50918   -27.639172   -8.4015665]
2 [ 34.66662  -27.38856   -8.202412]
3 [-35.01347  -28.389769  -9.379036]
4 [ 35.357594 -28.340115  -8.737451]
5 [ 35.06292  -28.791862  -7.920139]
6 [ 35.12232  -28.025568  -8.438357]
7 [-34.709858  -28.492348   -7.9871583]
8 [-34.16562  -27.651974  -8.296957]
9 [-34.01467  -27.425991  -8.542728]
10 [-34.589207  -26.794699   -7.9599514]
11 [-33.7034   -27.556913  -6.724072]
12 [-32.954582  -27.075233   -7.6588087]
13 [-33.3348   -26.861336  -8.06112 ]
14 [-34.122303 -28.119959  -7.858933]
15 [-33.932766 -27.566809  -8.017298]
16 [-34.707756 -27.124765  -9.155902]
17 [ 34.312122 -26.923996  -8.485293]
18 [ 34.638252  -26.665678   -8.7566805]
19 [ 35.307434 -26.312263  -8.163395]
20 [-35.434082 -26.071924  -8.643723]
21 [-35.16003  -26.542374  -8.962957]
22 [-34.137714 -26.128155  -8.694891]
23 [ 35.15582  -25.390306  -9.582587]
24 [ 35.094425 -25.399876  -8.848779]
25 [ 35.143196 -25.233618  -7.952073]
26 [ 35

440 [ 14.03162  32.18879 -19.87018]
441 [ 13.549717  32.16074  -19.417015]
442 [ 13.688169  31.60896  -20.366716]
443 [ 13.252124  32.440136 -19.273724]
444 [ 13.187559  32.493828 -18.589493]
445 [ 13.406126  32.221577 -19.525557]
446 [ 12.242598  32.238697 -19.828537]
447 [ 12.497123  32.134647 -19.383125]
448 [ 12.444752  32.1978   -19.253143]
449 [ 11.603861  31.60675  -19.37369 ]
450 [ 11.478486  30.999014 -18.610115]
451 [ 11.707147  31.546986 -18.8102  ]
452 [ 10.32943   31.693756 -19.017036]
453 [ 10.0209255  31.23753   -18.691538 ]
454 [  9.848187  31.215618 -18.72216 ]
455 [  9.974084  31.121868 -17.786465]
456 [  9.901611  31.386173 -17.021662]
457 [ 10.106078  31.63142  -16.875061]
458 [ 10.534457  31.331596 -16.252213]
459 [ 10.372997  31.083494 -16.761646]
460 [  9.846072  30.887585 -16.254896]
461 [  9.814405  31.432154 -17.051037]
462 [ 10.173944  31.339632 -17.326582]
463 [ 10.301368  30.795002 -18.25556 ]
464 [ 11.210666  30.865768 -17.980743]
465 [ 10.134474  31.11128